In [1]:
import pandas as pd
from sklearn.cluster import AgglomerativeClustering

df = pd.read_csv("grouping.csv")

In [2]:
df = df.drop(columns=["id_x", "Unnamed: 0", "release_date", 'genres'])

In [6]:
test = df[:1000]
clustering = AgglomerativeClustering(n_clusters=100).fit(test)

In [7]:
clustering.labels_

array([3, 3, 7, 3, 2, 7, 9, 7, 1, 9, 9, 7, 3, 7, 7, 2, 9, 3, 3, 9, 2, 7,
       3, 7, 3, 7, 3, 2, 7, 7, 6, 7, 7, 3, 7, 7, 2, 2, 2, 2, 7, 7, 7, 7,
       2, 7, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 7, 7, 7, 7, 3, 2, 7, 2, 7, 7,
       2, 2, 7, 7, 2, 3, 2, 2, 3, 2, 7, 2, 9, 2, 7, 9, 2, 2, 3, 7, 7, 3,
       2, 5, 7, 7, 2, 2, 4, 6, 3, 2, 7, 2, 7, 7, 7, 3, 3, 2, 7, 2, 9, 2,
       2, 3, 2, 7, 2, 2, 4, 3, 7, 7, 7, 3, 2, 7, 7, 2, 9, 9, 7, 7, 3, 2,
       2, 7, 7, 2, 7, 2, 7, 2, 2, 7, 6, 2, 7, 3, 7, 7, 3, 9, 7, 2, 2, 2,
       7, 7, 1, 2, 2, 6, 7, 3, 9, 7, 7, 4, 3, 3, 3, 2, 3, 6, 3, 3, 3, 5,
       7, 4, 7, 7, 3, 2, 7, 5, 6, 9, 2, 7, 2, 3, 3, 2, 7, 7, 7, 2, 3, 7,
       7, 7, 7, 7, 3, 7, 3, 7, 3, 3, 9, 2, 7, 9, 2, 2, 9, 8, 2, 2, 3, 7,
       6, 4, 7, 3, 3, 2, 2, 3, 6, 6, 6, 7, 2, 6, 2, 2, 7, 3, 7, 1, 7, 6,
       2, 7, 2, 7, 2, 7, 1, 7, 7, 2, 3, 2, 6, 2, 7, 6, 9, 2, 7, 3, 3, 2,
       6, 3, 2, 4, 7, 6, 7, 4, 6, 7, 6, 7, 3, 7, 1, 3, 3, 7, 2, 3, 6, 8,
       6, 8, 2, 3, 7, 9, 9, 2, 1, 3, 4, 9, 7, 3, 9,

In [9]:
import numpy as np

In [10]:
test['group'] = np.array(clustering.labels_)

/tmp/ipykernel_337/2060270766.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['group'] = np.array(clustering.labels_)


In [23]:
test[test['explicit']==1]

,duration_ms,explicit,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,autor_code,group
175,703373,1,0.347,0.302,0,-12.104,0.0291,0.1630,0.000092,0.140,0.0332,105.702,332,5
384,227933,1,0.484,0.165,7,-18.568,0.0498,0.7280,0.000001,0.115,0.1930,159.174,824,9
555,271933,1,0.485,0.943,4,-4.423,0.0416,0.0639,0.179000,0.144,0.8890,141.982,333,6
